In [4]:
import provider
import datetime
import pytz

In [5]:
bmdp = provider.BitmexDataProvider(pytz.utc.localize(datetime.datetime.now()) - datetime.timedelta(days=2),
                                   pytz.utc.localize(datetime.datetime.now()))
df = await bmdp._ConvertFilesToDataFrames().asend(None)
await bmdp.Close()

In [ ]:
import datetime
import pandas as pd
# compute by the minute Open/High/Low/Close/Volume 
#df[df['symbol'] == 'XBTUSD']
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dD%H:%M:%S.%f')
df['date'] = df['timestamp'].map(lambda x:x.date())
df['hour'] = df['timestamp'].map(lambda x:x.hour)
df['minute'] = df['timestamp'].map(lambda x:x.minute)
df['second'] = df['timestamp'].map(lambda x:x.second)


In [ ]:
df[df['symbol'] == 'XBTUSD']

In [ ]:
xbtusd_df = df[df['symbol'] == 'XBTUSD']
group = xbtusd_df.groupby(['symbol','date', 'hour', 'minute'])


In [ ]:
# open/high/low/close/volume
ohlcv = group.agg({'price': ['first', 'max', 'min', 'last',], 'size': 'sum'})
ohlcv

In [ ]:
# spot check agg works correctly for minute OCHLV data
l = xbtusd_df[(xbtusd_df['hour'] == 0) & (xbtusd_df['minute'] == 0)]['price'].min()
h = xbtusd_df[(xbtusd_df['hour'] == 0) & (xbtusd_df['minute'] == 0)]['price'].max()
o = xbtusd_df[(xbtusd_df['hour'] == 0) & (xbtusd_df['minute'] == 0)]['price'].iloc[0]
c = xbtusd_df[(xbtusd_df['hour'] == 0) & (xbtusd_df['minute'] == 0)]['price'].iloc[-1]
v = xbtusd_df[(xbtusd_df['hour'] == 0) & (xbtusd_df['minute'] == 0)]['size'].sum()

(o, c, h, l, v)

In [ ]:
#ohlcv[['price', 'size']]
import datetime
ohlcv.loc[('XBTUSD', datetime.datetime.now().date() - datetime.timedelta(days=1), 0, 0)]

In [ ]:
all_group = df.groupby(['symbol','date', 'hour', 'minute'])
all_ohlcv = all_group.agg({'price': ['first', 'max', 'min', 'last',], 'size': 'sum'})


In [ ]:
dir(all_ohlcv.index)

In [ ]:
set(all_ohlcv.index.get_level_values(level=0))

In [ ]:
asset_details = await bmdp.GetAssetDetails(all_ohlcv)

In [ ]:
futures_df = pd.DataFrame()
for detail_data in asset_details.values():
    # futures sid is the futures_df.index
    futures_df = futures_df.append(
      pd.DataFrame(pd.Series(detail_data)).T, ignore_index=True).drop_duplicates()
futures_df = futures_df[['symbol', 'expiry']].rename_axis('sid')


In [ ]:
futures_df

In [ ]:
sid_fdf = futures_df
sid_fdf['sid'] = sid_fdf.index
a2 = all_ohlcv.copy()
a2.columns = a2.columns.droplevel()
res = a2.reset_index().merge(sid_fdf, on='symbol')


In [ ]:
res2 = res.set_index('date')

In [ ]:
grouped = res2.groupby('sid')
writables = []
for sid in grouped.groups.keys():
    writables.append((sid, grouped.get_group(sid)))
writables

In [ ]:
for group in grouped:
    print(group[0])

In [ ]:
grouped.groups.keys()